In [1]:
import os
import json
import ast

from dotenv import load_dotenv

from utils.googlemaps.api import gm_distance_api, chunks

# LOAD ENV VARIABLES

In [2]:
load_dotenv()

API_KEY_PATH = os.environ['API_KEY_PATH']
DATA_PATH = os.environ['DATA_PATH']

In [3]:
with open(os.path.join(API_KEY_PATH, 'api_key.json'), 'r') as file:
    API_KEY = json.load(file)

# LOAD CHECKPOINTS

In [4]:
with open(os.path.join(DATA_PATH, 'CHECKPOINTS', 'mandatory_checkpoints.json'), 'r') as file:
    mandatory_checkpoints = json.load(file)
    
mandatory_checkpoints = {k: ast.literal_eval(v) for k, v in mandatory_checkpoints.items()}

with open(os.path.join(DATA_PATH, 'CHECKPOINTS', 'target_coords.json'), 'r') as file:
    target_coords = json.load(file)
    
target_coords= {k: ast.literal_eval(v) for k, v in target_coords.items()}

In [5]:
checkpoints = mandatory_checkpoints.copy()
checkpoints.update(target_coords)

In [6]:
checkpoints

{'st_imier_haut': (47.158679, 6.992699),
 'st_imier_bas': (47.15127, 6.99283),
 'lenzerheide_haut': (46.72093, 9.530324),
 'lenzerheide_bas_lift': (46.724895, 9.547017),
 'lenzerheide_bas_sporz': (46.715323, 9.540237),
 'beckenried': (46.967084, 8.475212),
 'gersau': (46.990807, 8.525738),
 'goschenen': (46.665698, 8.588439),
 'sanetsch': (46.363102, 7.295129),
 'versoix': (46.295182, 6.167042),
 'mies': (46.296179, 6.168144),
 'st_maurice': (46.223444, 7.003184),
 'diablerets': (46.361971, 7.228706),
 'rougemont': (46.486861, 7.228648),
 'chatel_st_denis': (46.512836, 6.911657),
 'vaumarcus': (46.870955, 6.754053),
 'chaux_de_fond': (47.122309, 6.885278),
 'chasseral': (47.125755, 7.038184),
 'delemont': (47.375549, 7.3615),
 'liesberg': (47.397281, 7.400262),
 'liesberg_2': (47.402422, 7.443518),
 'basel': (47.535692, 7.609889),
 'kaiseraugst': (47.536199, 7.721017),
 'kaiserstuhl': (47.564849, 8.420254),
 'aadorf': (47.494296, 8.894507),
 'wil': (47.466492, 9.029698),
 'herisau': (4

# GET API DATA

In [8]:
api_data_time = {}
api_data_dist = {}

for cp, coord in checkpoints.items():
    
    destinations = checkpoints.copy()
    destinations.pop(cp)
    destinations = [x for x in destinations.values()]
    
    time = []
    distance = []
    if len(destinations) > 25:
        
        for chunk_destinations in chunks(destinations, 25):
            
            data = gm_distance_api(coord, chunk_destinations, API_KEY)
            time.extend(data['time'])
            distance.extend(data['distance'])
            
    else:
        
        data = gm_distance_api(coord, destinations, API_KEY)
        time.extend(data['time'])
        distance.extend(data['distance'])
    
    api_data_time[data['origin_addresses'][0]] = time
    api_data_dist[data['origin_addresses'][0]] = distance

# SAVE DATA

In [9]:
with open(os.path.join(DATA_PATH, 'DIST_MATRIX', 'api_data_dist_05.json'), 'w') as outfile:
    json.dump(api_data_dist, outfile)
    
with open(os.path.join(DATA_PATH, 'DIST_MATRIX', 'api_data_time_05.json'), 'w') as outfile:
    json.dump(api_data_time, outfile)
    
with open(os.path.join(DATA_PATH, 'DIST_MATRIX', 'checkpoints_05.json'), 'w') as outfile:
    json.dump(checkpoints, outfile)